In [1]:
pip install python-edgar

Note: you may need to restart the kernel to use updated packages.


In [72]:
# Import packages
import edgar
import pandas as pd
import requests
import json

### Downlaod Master Indices

In [73]:
# Download master indices from SEC
user_agent = "DRod/1.0 (dr1902@nyu.edu; For educational purposes)"
edgar.download_index('.', 2022, user_agent, skip_all_present_except_last=False)

### Retrieve the latest S&P500 composition

In [74]:
# Load historical sp500 composition
sp500_com = pd.read_excel('sp500_composition.xlsx')

In [75]:
# Present year
today = pd.to_datetime('today').year

# Retrieve the sp500 composition from the previous quarter
def update_indices(year, quarter, form):
    # Assign timestamps for different quarters
    if quarter == 1:
        date = pd.Timestamp(year, 1, 1)
    elif quarter == 2:
        date = pd.Timestamp(year, 4, 1)
    elif quarter == 3:
        date = pd.Timestamp(year, 7, 1)
    else:
        date = pd.Timestamp(year, 10, 1)

    # Find the closest date to the assigned dates to the quarters
    closest_date = find_close_date(date)

    # Export company names of S&P500 based on the closest date
    if form == 'name':
        companies = sp500_com.loc[sp500_com.Date==closest_date, 'Company Name']
        lst = []
        for company in companies:
            lst.append(company.strip())
        return lst
    # Export tickers of S&P500 based on the closest date
    elif form == 'ticker':
        tickers = sp500_com.loc[sp500_com.Date==closest_date, 'Ticker']
        lst = []
        for ticker in tickers:
            lst.append(ticker.strip())
        return lst


# Find the closest date to the assigned dates to the quarters
def find_close_date(date):
    
    close_date = pd.Timestamp(2000, 1, 1)
    time_delta = date - close_date
    
    for row in sp500_com.iterrows():
        if row[1][0] <= date and date - row[1][0] < time_delta:
            close_date = row[1][0]
            time_delta = date - close_date
    return close_date

In [76]:
# URL to the SEC's CIK-to-Ticker mapping file
url = "https://www.sec.gov/files/company_tickers.json"
headers = {'User-Agent':"DRod/1.0 (dr1902@nyu.edu; For educational purposes)"}

# Download the file
response = requests.get(url, headers=headers)
data = response.json()

# Function to get ticker from CIK
def get_ticker_from_cik(cik):
    for company in data.values():
        if company['cik_str'] == cik:
            return company['ticker']
    return None

In [77]:
prefix = 'https://www.sec.gov/Archives/'
company_name = []
time = []
file = []
link = []

for year in range(2022, today+1):
    
    # There's been only 3 quarters so far in 2024
    if year==today:
        quarters = 4
    else:
        quarters = 5
    
    for quarter in range(1, quarters):
        
        # Loading master index
        filename = str(year)+'-QTR'+str(quarter)+'.tsv'
        mas_ind = pd.read_csv(filename, sep='|', header=None)
        
        # Retrieving tickers using the updated(latest) sp500 composition
        sp_tlst = update_indices(year, quarter, 'ticker')
        
        # Checking every file entry
        for i in range(len(mas_ind.loc[:, 0])):

            # Getting tickers using cik from master index
            cik = int(mas_ind.loc[i, 0])
            ticker = get_ticker_from_cik(cik)

            # Checking if the ticker is in sp500
            if ticker in sp_tlst:
                # Checking if the file is 10K or 10Q
                if mas_ind.loc[i, 2] == '10-K' or mas_ind.loc[i, 2]=='10-K/A' or mas_ind.loc[i, 2]=='10-Q' or mas_ind.loc[i, 2]=='10-Q/A':
                    company_name.append(mas_ind.loc[i, 1])
                    time.append(str(year)+'Q'+str(quarter))
                    file.append(mas_ind.loc[i, 2])
                    url = prefix+mas_ind.loc[i, 5]
                    link.append(url)
                else:
                    continue
            else:
                continue

/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_96250/2795958831.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1][0] <= date and date - row[1][0] < time_delta:
/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_96250/2795958831.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close_date = row[1][0]
/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_96250/2795958831.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by po

In [78]:
file_dict = {'company_name': company_name, 'published_time': time, 'type': file, 'link_url': link}
df = pd.DataFrame(file_dict)
df

,company_name,published_time,type,link_url
0,HENRY SCHEIN INC,2022Q1,10-K,https://www.sec.gov/Archives/edgar/data/100022...
1,WATERS CORP /DE/,2022Q1,10-K,https://www.sec.gov/Archives/edgar/data/100069...
2,ESTEE LAUDER COMPANIES INC,2022Q1,10-Q,https://www.sec.gov/Archives/edgar/data/100125...
3,"NetApp, Inc.",2022Q1,10-Q,https://www.sec.gov/Archives/edgar/data/100204...
4,AMEREN CORP,2022Q1,10-K,https://www.sec.gov/Archives/edgar/data/100291...
...,...,...,...,...
5351,SYSCO CORP,2024Q3,10-K,https://www.sec.gov/Archives/edgar/data/96021/...
5352,TELEFLEX INC,2024Q3,10-Q,https://www.sec.gov/Archives/edgar/data/96943/...
5353,"TERADYNE, INC",2024Q3,10-Q,https://www.sec.gov/Archives/edgar/data/97210/...
5354,TEXAS INSTRUMENTS INC,2024Q3,10-Q,https://www.sec.gov/Archives/edgar/data/97476/...


In [79]:
df.to_csv('sp500_files.csv', index=False)